**Translation** menerjemahkan urutan teks dari satu bahasa ke bahasa lain.

**Panduan:**
1. Melakukan ine-tune pada model T5 yang telah di-pretrain untuk menerjemahkan bahasa Inggris ke Bahasa Prancis.
2. Menggunakan model yang telah difine-tune untuk inferensi.



In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load Dataset

Mulai dengan load English-French subset dari dataset OPUS Books

In [ ]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Split dataset

In [ ]:
books = books["train"].train_test_split(test_size=0.2)

In [ ]:
books["train"][0]
{'id': '90560',
 'translation': {'en': 'But this lofty plateau measured only a few fathoms, and soon we reentered Our Element.',
  'fr': 'Mais ce plateau élevé ne mesurait que quelques toises, et bientôt nous fûmes rentrés dans notre élément.'}}

{'id': '90560',
 'translation': {'en': 'But this lofty plateau measured only a few fathoms, and soon we reentered Our Element.',
  'fr': 'Mais ce plateau élevé ne mesurait que quelques toises, et bientôt nous fûmes rentrés dans notre élément.'}}

# Preprocessing

Load T5 tokenizer untuk memproses pasangan bahasa English-French:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

**Fungsi pra-pemrosesan yang diperlukan:**



1.   Memberi awalan pada masukan dengan sebuah prompt sehingga T5 tahu bahwa ini adalah tugas terjemahan.
2.   Tokenisasi masukan (Inggris) dan target (Prancis) secara terpisah
3. Memotong urutan sehingga tidak lebih panjang dari panjang maksimum yang ditetapkan oleh parameter max_length.



In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Untuk menerapkan fungsi *preprocess_function* pada seluruh dataset, gunakan metode *map* dari library *Datasets*. (Fungsi map dapat dipercepat dengan mengatur batched=True):

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

 Buatlah sebuah batch contoh menggunakan ***DataCollatorForSeq2Seq***

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Evaluate

Gunakan library Evaluate untuk load **metric SacreBLEU**, yaitu salah satu metric yang digunakan untuk evaluasi performa model.

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Train

Load model T5 dengan ***AutoModelForSeq2SeqLM:***

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Langkah yang tersisa adalah:**


1.   Tentukan hiperparameter pelatihan dalam Seq2SeqTrainingArguments. Satu-satunya parameter yang diperlukan adalah output_dir yang menentukan di mana menyimpan model. Lalu unggah model ini ke Hub dengan mengatur *push_to_hub=True*. Pada akhir setiap epoch, Trainer akan mengevaluasi metrik SacreBLEU dan menyimpan checkpoint pelatihan.
2.   Siapkan argumen training ke *Seq2SeqTrainer *bersama dengan model, dataset, tokenizer, data collator, dan fungsi compute_metrics.
3. Panggil train() untuk melakukan fine-tuning pada model.


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.860900,1.633250,5.515200,17.592200
2,1.815500,1.609982,5.686600,17.578400


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=12710, training_loss=1.8739956235247837, metrics={'train_runtime': 2852.4332, 'train_samples_per_second': 71.285, 'train_steps_per_second': 4.456, 'total_flos': 5001965470875648.0, 'train_loss': 1.8739956235247837, 'epoch': 2.0})

Selanjutnya share model ke Hub dengan perintah *push_to_hub()* agar orang lain bisa menggunakan modelnya:

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1706508702.d07e29dc856e.472.0:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Indah1/my_awesome_opus_books_model/commit/fe3513bd40bfe45b43b5371d6bfd2017d1d57959', commit_message='End of training', commit_description='', oid='fe3513bd40bfe45b43b5371d6bfd2017d1d57959', pr_url=None, pr_revision=None, pr_num=None)

# Inference

Prefix inputan:


In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="my_awesome_opus_books_model")
translator(text)
[{'translation_text': 'Legumes partagent des ressources avec des bactéries azotantes.'}]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1049: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'Legumes partagent des ressources avec des bactéries azotantes.'}]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

"Les lignées partagent les ressources avec les bactéries de fixation d'azote."